In [2]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import random 
import collections
# use multiprocessing for multicore ops
import copy

In [3]:
data2 = pd.read_csv("../data/questions.csv", delimiter=',', encoding="utf-8-sig")
data1 = copy.deepcopy(data2[:50000])
print(data1.shape)
print(data1.iloc[0:1,3:])
#random.shuffle(data1)
data1.dropna(inplace=True)

(50000, 6)
                                           question1  \
0  What is the step by step guide to invest in sh...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  


In [5]:
X = data1[["question1","question2"]]
y = data1[['is_duplicate']]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
#nltk.download()
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english")) 


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [111]:
def question_to_words( raw_quest, lower=True, remove_not = False ):
    
    lemmetizer = nltk.WordNetLemmatizer()
    
    letters_only = re.sub("[^a-zA-Z]", " ", raw_quest)
    if lower==True:
        letters_only = letters_only.lower()
    words = nltk.word_tokenize(letters_only)
    #words = nltk.word_tokenize(s)
    stops = set(stopwords.words("english")) 
    not_stop = ['no', 'nor', 'not', "don't", "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    if remove_not == True:
        #stop = stops.difference(not_stop)
        stops = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']
    meaningful_words = [w for w in words if not w in stops]
    
    meaningful_words = [lemmetizer.lemmatize(w) for w in meaningful_words]
    return( " ".join( meaningful_words ))

In [116]:
#X_train['question2']= X_train['question2'].apply(question_to_words)
#X_train = X_train.apply(question_to_words)
print(X_train.shape)

#a = X_train[['question1']]
#a = a.apply(question_to_words)
X_train['question1'] = X_train['question1'].apply(question_to_words)
X_train['question2'] = X_train['question2'].apply(question_to_words)
#b=X_train[['question2']]
#b.apply(question_to_words)
X_train


(33500, 2)


/home/ss/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/ss/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,question1,question2
23990,get velocity equation dt dv f mg kv,get velocity equation math displaystyle int ma...
8729,im mechanical student know autocad creo ansys ...,started nd year mechanical engineering know ca...
3451,could one punch man kill wolverine deadpool,would win one one fight saitama one punch man ...
2628,sell source code app,sell source code
38352,abortion legal case involve rape incest possib...,abortion allowed except case low chance surviv...
35138,much get donate kidney,donate kidney
37071,video chat facebook mobile device,video chat facebook mobile
32984,good book management market,best book sale marketing entrepreneurship
24806,happens eat week,happens body eat day
24863,best begging sign seen,best travel content ever seen


In [130]:
print(X_train.shape)
print(X_train.head)

(33500, 2)
<bound method NDFrame.head of                                                question1  \
23990                get velocity equation dt dv f mg kv   
8729   im mechanical student know autocad creo ansys ...   
3451         could one punch man kill wolverine deadpool   
2628                                sell source code app   
38352  abortion legal case involve rape incest possib...   
35138                             much get donate kidney   
37071                  video chat facebook mobile device   
32984                        good book management market   
24806                                   happens eat week   
24863                             best begging sign seen   
10245                       biggest lie ever told people   
13143                           part time work done home   
35126         ionization energy hydrogen atom determined   
49799                bitcoin technical flaw mt gox issue   
42487                                        th commerce   

In [117]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

Creating the bag of words...



In [52]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
#add whitelisting
# use ngrams , ngram_range=(1,2) check for various combinations

In [118]:
#corpus = X_train["question1"] + " " + X_train["question2"]
corpus = X_train["question1"] + " " + X_train["question2"]
cv = vectorizer.fit(corpus)

cv
from sklearn.metrics.pairwise import cosine_similarity


In [131]:
print(X_train)
print(y_train)

                                               question1  \
23990                get velocity equation dt dv f mg kv   
8729   im mechanical student know autocad creo ansys ...   
3451         could one punch man kill wolverine deadpool   
2628                                sell source code app   
38352  abortion legal case involve rape incest possib...   
35138                             much get donate kidney   
37071                  video chat facebook mobile device   
32984                        good book management market   
24806                                   happens eat week   
24863                             best begging sign seen   
10245                       biggest lie ever told people   
13143                           part time work done home   
35126         ionization energy hydrogen atom determined   
49799                bitcoin technical flaw mt gox issue   
42487                                        th commerce   
41055        possible get custom specifi

In [136]:
#new_data = X_train + y_train
new_data = pd.concat([X_train, y_train], axis=1, join_axes=[X_train.index])
print(new_data.head)


<bound method NDFrame.head of                                                question1  \
23990                get velocity equation dt dv f mg kv   
8729   im mechanical student know autocad creo ansys ...   
3451         could one punch man kill wolverine deadpool   
2628                                sell source code app   
38352  abortion legal case involve rape incest possib...   
35138                             much get donate kidney   
37071                  video chat facebook mobile device   
32984                        good book management market   
24806                                   happens eat week   
24863                             best begging sign seen   
10245                       biggest lie ever told people   
13143                           part time work done home   
35126         ionization energy hydrogen atom determined   
49799                bitcoin technical flaw mt gox issue   
42487                                        th commerce   
41055     

In [137]:
duplicates = new_data[new_data.is_duplicate==1]
nonduplicates = new_data[new_data.is_duplicate==0]
print(duplicates.shape)
print(nonduplicates.shape)

(12482, 3)
(21018, 3)


In [138]:
duplicates.head

<bound method NDFrame.head of                                                question1  \
37071                  video chat facebook mobile device   
13143                           part time work done home   
42487                                        th commerce   
19221                                      find gmail id   
3354                                  happened flight mh   
9116                 india defeat china full fledged war   
48412                                hire ethical hacker   
31911                       build self esteem confidence   
16540                       concentrate better way study   
47527          human eating placenta legally cannibalism   
35134                           good english song listen   
7840                                 legal sex car india   
14454     white american think white european vice versa   
2071                saltwater taffy candy imported japan   
9358                                person control anger   
19826     

In [144]:
def getSimilarity(df):
    r,c = df.shape
    print(str(r)+ str(c))
    final_vec = []
    for j in range(r):
        a = cv.transform([df.iloc[j].question1])
        b = cv.transform([df.iloc[j].question2])
        sim = round(cosine_similarity(a,b).ravel()[0], 3)
        c = df.iloc[j].is_duplicate
        final_vec.append([sim, c])
    return final_vec


In [145]:
sim = getSimilarity(new_data)

335003


In [146]:
sim

[[0.555, 0],
 [0.306, 0],
 [0.481, 0],
 [0.866, 0],
 [0.555, 0],
 [0.707, 0],
 [0.894, 1],
 [0.224, 0],
 [0.577, 0],
 [0.516, 0],
 [0.8, 0],
 [1.0, 1],
 [0.866, 0],
 [0.25, 0],
 [1.0, 1],
 [0.0, 0],
 [0.707, 0],
 [0.612, 0],
 [0.833, 0],
 [0.289, 1],
 [0.632, 0],
 [1.0, 0],
 [0.154, 0],
 [0.707, 1],
 [0.408, 0],
 [0.183, 0],
 [0.894, 1],
 [1.0, 0],
 [0.667, 1],
 [0.889, 0],
 [0.894, 0],
 [0.775, 1],
 [0.5, 0],
 [0.5, 0],
 [0.447, 1],
 [0.408, 1],
 [0.5, 1],
 [0.0, 0],
 [0.894, 1],
 [0.385, 1],
 [0.612, 0],
 [0.894, 1],
 [0.0, 0],
 [0.0, 0],
 [0.667, 1],
 [0.0, 0],
 [0.577, 1],
 [0.0, 0],
 [0.0, 0],
 [0.0, 0],
 [0.387, 0],
 [0.204, 1],
 [0.516, 1],
 [0.167, 0],
 [0.289, 0],
 [1.0, 1],
 [0.333, 1],
 [0.447, 0],
 [0.671, 1],
 [0.378, 0],
 [0.105, 0],
 [0.577, 0],
 [0.57, 0],
 [0.866, 0],
 [0.667, 0],
 [0.516, 0],
 [0.0, 0],
 [0.9, 0],
 [0.5, 1],
 [0.816, 1],
 [1.0, 1],
 [0.267, 0],
 [0.258, 1],
 [0.577, 1],
 [0.258, 0],
 [0.866, 0],
 [0.0, 0],
 [0.875, 0],
 [0.333, 0],
 [0.73, 1],
 [0.707

In [150]:
df1 = pd.DataFrame(sim)
df1.columns = ["cos_sim", "y"]
df1


,cos_sim,y
0,0.555,0
1,0.306,0
2,0.481,0
3,0.866,0
4,0.555,0
5,0.707,0
6,0.894,1
7,0.224,0
8,0.577,0
9,0.516,0


In [165]:
X = df1['cos_sim']
y = df1['y']


In [167]:
from sklearn.ensemble import RandomForestClassifier
clf= RandomForestClassifier()
X = X.values.reshape(1, -1)
y = y.values.reshape(1, -1)
clf.fit(X,y)


/home/ss/.local/lib/python3.5/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)